# Understanding Lego sets popularity

Now let's now move on to the competition and challenge.

## 📖 Background
You recently applied to work as a data analyst intern at the famous Lego Group in Denmark. As part of the job interview process, you received the following take-home assignment:

You are asked to use the provided dataset to understand the popularity of different Lego sets and themes. The idea is to become familiarized with the data to be ready for an interview with a business stakeholder.

<img src="http://oyster.ignimgs.com/wordpress/stg.ign.com/2014/02/LEGOHeader2.jpg">

## 💾 The data

#### You received access to a database with the following tables. You can also see above a visualization of how the tables are related to each other. ([source](https://rebrickable.com/downloads)):

#### inventory_parts
- "inventory_id" - id of the inventory the part is in (as in the inventories table)
- "part_num" - unique id for the part (as in the parts table)
- "color_id" - id of the color
- "quantity" - the number of copies of the part included in the set
- "is_spare" - whether or not it is a spare part

#### parts
- "part_num" - unique id for the part (as in the inventory_parts table)
- "name" - name of the part
- "part_cat_id" - part category id (as in part_catagories table)

#### part_categories
- "id" - part category id (as in parts table)
- "name" - name of the category the part belongs to

#### colors
- "id" - id of the color (as in inventory_parts table)
- "name" - color name
- "rgb" - rgb code of the color
- "is_trans" - whether or not the part is transparent/translucent

#### inventories
- "id" - id of the inventory the part is in (as in the inventory_sets and inventory_parts tables)
- "version" - version number
- "set_num" - set number (as in sets table)

#### inventory_sets
- "inventory_id" - id of the inventory the part is in (as in the inventories table)
- "set_num" - set number (as in sets table)
- "quantity" - the quantity of sets included

#### sets
- "set_num" - unique set id (as in inventory_sets and inventories tables)
- "name" - the name of the set
- "year" - the year the set was published
- "theme_id" - the id of the theme the set belongs to (as in themes table)
- num-parts - the number of parts in the set

#### themes
- "id" - the id of the theme (as in the sets table)
- "name" - the name of the theme
- "parent_id" - the id of the larger theme, if there is one


***Acknowledgments**: Rebrickable.com*

![erd](data/lego_erd.png)

In [28]:
# This is a chart, switch to the DataCamp editor to view and configure it.

,year,sets_per_year
0,1950,7
1,1953,4
2,1954,14
3,1955,28
4,1956,12
...,...,...
61,2013,593
62,2014,713
63,2015,665
64,2016,596


## 💪 Challenge
Create a report to summarize your findings. Include:

1. What is the average number of Lego sets released per year?
2. What is the average number of Lego parts per year?
3. Create a visualization for item 2.
4. What are the 5 most popular colors used in Lego parts?
5. [Optional] What proportion of Lego parts are transparent?
6. [Optional] What are the 5 rarest lego bricks?
7. Summarize your findings.

## 💡 Learn more

The following DataCamp courses can help review the skills needed for this challenge:

* [Introduction to SQL](https://www.datacamp.com/courses/introduction-to-sql)
* [Intermediate SQL](https://www.datacamp.com/courses/intermediate-sql)
* [Joining Data in SQL](https://www.datacamp.com/courses/joining-data-in-sql)

## ✅ Checklist before publishing
- Rename your workspace to make it descriptive of your work. N.B. you should leave the notebook name as notebook.ipynb.
- **Remove redundant cells** like the introduction to data science notebooks, so the workbook is focused on your story.
- Check that all the cells run without error.

## ⌛️ Time is ticking. Good luck!

In [5]:
-- 1. What is the average number of Lego sets released per year?

SELECT avg(count) as lego_sets_released_per_year
    FROM (
        SELECT COUNT(d.set_num) as Count,
        d.set_num,
        d.year
        FROM inventory_parts a
        INNER JOIN inventories b 
            ON a.inventory_id = b.id
        INNER JOIN inventory_sets c
            ON b.id = c.inventory_id
        INNER JOIN sets d
            ON b.set_num = d.set_num
        Group by d.year, d.set_num
    ) as counts

,lego_sets_released_per_year
0,8.470588


In [61]:
 SELECT  e.part_num,
                COUNT(e.part_num) OVER (PARTITION BY d.year) as count,
                d.year
                FROM inventory_parts a
                INNER JOIN inventories b 
                    ON a.inventory_id = b.id
                INNER JOIN sets d
                    ON b.set_num = d.set_num
                INNER JOIN parts e 
                    ON a.part_num = e.part_num
                GROUP BY e.part_num, d.year
Limit 20;

,part_num,count,year
0,bwindow01,6,1950
1,bwindow03,6,1950
2,bslot04,6,1950
3,bwindow02,6,1950
4,bdoor01,6,1950
5,bslot02,6,1950
6,bwindow01,6,1953
7,bslot03,6,1953
8,bslot01,6,1953
9,bdoor01,6,1953


In [64]:
-- 2. What is the average number of Lego parts per year?
SELECT sum(count_partition)/count(distinct(year_partition)) as lego_parts_per_year
    FROM (
        SELECT  e.part_num,
                COUNT(e.part_num) OVER (PARTITION BY d.year) as count_partition,
                d.year as year_partition
                FROM inventory_parts a
                INNER JOIN inventories b 
                    ON a.inventory_id = b.id
                INNER JOIN sets d
                    ON b.set_num = d.set_num
                INNER JOIN parts e 
                    ON a.part_num = e.part_num
                GROUP BY e.part_num, d.year
        ) as counts

,lego_parts_per_year
0,2.460838e+06


In [36]:
-- 2. What is the average number of Lego parts per year?
SELECT avg(count) as lego_parts_per_year
    FROM (
        SELECT  e.part_num,
                COUNT(e.part_num) OVER (PARTITION BY d.year) as count,
                d.year
                FROM inventory_parts a
                INNER JOIN inventories b 
                    ON a.inventory_id = b.id
                INNER JOIN sets d
                    ON b.set_num = d.set_num
                INNER JOIN parts e 
                    ON a.part_num = e.part_num
                GROUP BY e.part_num, d.year
        ) as counts

,lego_parts_per_year
0,2329.236028
